# Aufgabe 10: 3-SAT Problem

In [ ]:
import numpy as np
rng = np.random.default_rng()

################    Parameter   ################

anzahlVariablen = 20
anzahlKlauseln = 30
maxLiteraleProKlausel = 3

maxGenerationen = 30
populationsGroesse = 100
anzahlSelektion = 20
anzahlMutation = 20
anzahlRekombination = 30 #muss eine Gerade Zahl sein
anzahlNeueIndividuen = 30

################################################

In [ ]:
def createRandomKNF():
    Klauselmenge = []
    for i in range(anzahlKlauseln):
        numLiterale = rng.integers(low=1, high=(maxLiteraleProKlausel + 1))
        Klausel = []
        for k in range(numLiterale):
            literal = rng.integers(low = 1, high=(anzahlVariablen + 1))
            literal = -1*literal if rng.integers(low=0, high=2) == 1 else literal
            while(literal in Klausel or -1*literal in Klausel):
                literal = rng.integers(low = 1, high=(anzahlVariablen + 1))
                literal = -1*literal if rng.integers(low=0, high=2) == 1 else literal
            Klausel.append(literal)
        for k in range(maxLiteraleProKlausel - numLiterale):
            Klausel.append(0)
        Klauselmenge.append(Klausel)

    return Klauselmenge
    

#alt:
#for i in range(25):
    #numLiterale = rng.integers(low=1, high=4)
    #if numLiterale == 1:
        #Klauselmenge.append([rng.choice(np.concatenate((np.arange(-20, 0), np.arange(1, 21)))), 0, 0])
    #if numLiterale == 2:
        #Klauselmenge.append([rng.choice(np.concatenate((np.arange(-20, 0), np.arange(1, 21)))), rng.choice(np.concatenate((np.arange(-20, 0), np.arange(1, 21)))), 0])
    #if numLiterale == 3:
        #Klauselmenge.append([rng.choice(np.concatenate((np.arange(-20, 0), np.arange(1, 21)))), rng.choice(np.concatenate((np.arange(-20, 0), np.arange(1, 21)))), rng.choice(np.concatenate((np.arange(-20, 0), np.arange(1, 21))))])



In [ ]:
knf = createRandomKNF()

In [ ]:
def InitPopulation():
    initPopulation = rng.integers(size=(populationsGroesse, (anzahlVariablen + 1)), low=0, high=2)
    for individuum in initPopulation:
        individuum[0] = 1

    return initPopulation

In [ ]:
def sortPopulation(population):
    bewertung = {}
    for i, individuum in enumerate(population):
        erfüllteKlauseln = 0
        for klausel in knf:
            for variable in klausel:
                if variable > 0 and individuum[variable] == 1:
                    erfüllteKlauseln = erfüllteKlauseln + 1
                    break
                elif variable < 0 and individuum[-1 * variable] == 0:
                    erfüllteKlauseln = erfüllteKlauseln + 1
                    break
        bewertung[i] = erfüllteKlauseln

    sortierteIndividuen = dict(sorted(bewertung.items(), key=lambda item: item[1]))

    return sortierteIndividuen

In [ ]:
def Selektion(sortedPopulation, population):
    selected = []
    np.array(selected)
    for i in range(anzahlSelektion):
        individuum = population[list(sortedPopulation.keys())[(populationsGroesse - 1)-i]]
        #print(individuum)
        selected.extend(individuum)
        
        #selected = np.concatenate((selected,individuum ), axis=0)
        #selected[99-i] = [individuum]
        #selectet = selected + individuum
        #selected
    selected = np.reshape(selected, (anzahlSelektion, (anzahlVariablen + 1)))
    
    return selected

In [ ]:
def Mutation(sortedPopulation, population):
    mutatedPopulation = []
    for i in range(anzahlMutation):
        individuum = population[list(sortedPopulation.keys())[(populationsGroesse - 1)-i] ]
        mutationSpot = rng.integers(low=1, high=(anzahlVariablen + 1))
        individuum[mutationSpot] = 1 if individuum[mutationSpot] == 0 else 0
        mutatedPopulation.append(individuum)

    return mutatedPopulation
        

In [ ]:
def Rekombination(sortedPopulation, population):
    recombinedPopulation = []
    for i in range(int(anzahlRekombination / 2)):
        individuum1 = np.array_split(population[list(sortedPopulation.keys())[(populationsGroesse - 1)-i]], 2)
        individuum2 = np.array_split(population[list(sortedPopulation.keys())[rng.integers(low=1, high=21)]], 2)

        new1 = np.concatenate((individuum1[0], individuum2[1]))
        new2 = np.concatenate((individuum2[0], individuum1[1]))

        recombinedPopulation.append(new1)
        recombinedPopulation.append(new2)
        
    return recombinedPopulation

In [ ]:
def createNewRandom():
    newRandoms = rng.integers(size=(anzahlNeueIndividuen, (anzahlVariablen+1)), low=0, high=2)
    for individuum in newRandoms:
        individuum[0] = 1
        
    return newRandoms

In [ ]:
def Evolution(iterations, startPopulation):
    population = startPopulation
    for i in range(iterations):
        sortedPopulation = sortPopulation(population)

        print(list(sortedPopulation.values())[populationsGroesse - 1])

        selektion = Selektion(sortedPopulation, population)
        mutation = Mutation(sortedPopulation, population)
        rekombination = Rekombination(sortedPopulation, population)
        newRandoms = createNewRandom()

        newPopulation = np.concatenate((selektion, newRandoms,mutation,rekombination ),  axis=0)

        population = newPopulation

In [ ]:
startPopulation = InitPopulation()
Evolution(maxGenerationen, startPopulation)